# Outline

- train word embeddings by using `matichon.json`
- use gensim `Word2Vec` model
- find similar words, calculate cosine similarity

# Import 

In [1]:
import pandas as pd
import numpy as np
import re, emoji, urllib, html
from gensim.models import Word2Vec
from pythainlp.tokenize import word_tokenize

- custom tokenization function
- remove all quatations and shrink newlines `\n` and white spaces 

In [2]:
def my_tokenize(text):
    ### REMOVE URL ###
    text = html.unescape(urllib.parse.unquote(text)) # unescape for unicode, unquote for escaped URL
    text = re.sub(r'https?.+?(?:\s|$)', '', text) # remove URL link
    ### REMOVE EMOJI ###
    text = emoji.replace_emoji(text) # remove emoji
    ### REPLACE ###
    text = re.sub(r'[“”„\"]', '', text) # remove double quotations
    text = re.sub(r'[‘’′′′′`\']', '', text) # remove single quotations
    text = re.sub(r'[\n\t\u00a0\xa0\u3000\u2002-\u200a\u202f]+', ' ', text) # shrink whitespaces e.g. good  boy -> good boy
    text = re.sub(r'[\r\u200b\ufeff]+', '', text) # remove non-breaking space
    text = re.sub(r'เเ', 'แ', text)
    ### SHRINK SOME REDUPLICATION ###
    text = re.sub(r'าา+', 'า', text)
    text = re.sub(r'ยย+', 'ย', text)
    text = re.sub(r'ๆๆ+', 'ๆ', text)
    text = re.sub(r'ะะ+', 'ะ', text)
    ### am ###
    text = re.sub(r'ํา','ำ', text) # o + า -> ำ
    text = re.sub(r'\u0E33([\u0E48\u0E49\u0E4A\u0E4B])', r'\1'+'\u0E33', text) # am + tone -> tone + am
    ### TOKENIZE AND FILTERING ###
    tokens = word_tokenize(text, keep_whitespace=False)
    tokens = [token.strip('(').strip(')') for token in tokens] # remove parenthesis sticked to token
    tokens = [token for token in tokens if re.match(r'[0-9A-zก-ไ][0-9A-zก-๙\.\-]*', token)] # remove non-word
    return tokens

# Load data

- need only article, drop the other columns
- tokenize article and store as list of tokens

In [3]:
df = pd.read_json('data/matichon.json')
df

,headline,article,date,category,url,id
0,ซาอุฯจ่อเปิดไฟเขียวให้สิทธิหญิงม่าย-หย่าร้างปก...,(2 ธ.ค.58) หนังสือพิมพ์อัล ริยาดของทางการซาอุด...,2015-12-04 03:35:18,foreign,https://www.matichon.co.th/foreign/news_293,293
1,"""ไก่อู""ชี้ ตู่-เต้น ไม่ได้มีหน้าที่ตรวจสอบทุจร...","""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",2015-12-04 04:10:49,politics,https://www.matichon.co.th/politics/news_329,329
2,"เปิดใจ ""โบว์ แวนดา"" ระหว่างรอยิ้มได้เต็มที่ในว...",แม้จะทำหน้าที่ภรรยาที่ดีมาเฝ้าปอ – ทฤษฎี สหวงษ...,2015-12-04 06:30:11,entertainment,https://www.matichon.co.th/entertainment/news_375,375
3,"""นาย ณภัทร"" ปลื้มคนชมแชมป์ขึ้นปกนิตยสารแห่งปี ...",กลายเป็นดาราหนุ่มเนื้อหอมแฟนคลับแน่น กระแสมาแร...,2015-12-04 07:10:26,entertainment,https://www.matichon.co.th/entertainment/news_393,393
4,"คอแทบหัก! แม่ยกแห่คล้องพวงมาลัยักษ์ ""บอย ศิริช...",แสดงดีจนเป็นที่ถูกอกถูกใจแฟนคลับ จนได้รับพวงมา...,2015-12-05 05:26:20,entertainment,https://www.matichon.co.th/entertainment/news_445,445
...,...,...,...,...,...,...
17104,โบว์ ณัฏฐา แจ้งความ พล.ต.อ.ศรีวราห์ ถูกพาดพิง...,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,2018-07-05 13:25:45,politics,https://www.matichon.co.th/politics/news_1029607,1029607
17105,ภาพบรรยากาศ ขุดทางระบายน้ำ เร่งนำ 13 ชีวิตออกจ...,วันที่ 5 กรกฎาคม เจ้าหน้าที่ขุดทางระบายน้ำที่ด...,2018-07-05 13:33:10,region,https://www.matichon.co.th/region/news_1029619,1029619
17106,สนช.ผ่านพ.ร.บ.สงฆ์ 3 วาระรวด พระมหากษัตริย์ทรง...,"สนช.ผ่าน พ.ร.บ.สงฆ์ 3 วาระรวด ""วิษณุ"" แจงสาระส...",2018-07-05 13:33:27,politics,https://www.matichon.co.th/politics/news_1029636,1029636
17107,นานาทรรศนะเพิ่มค่าปรับหมอ 5ล้านบ. สกัดเบี้ยว...,หมายเหตุ – จากกรณีที่ กระทรวงศึกษาธิการ (ศธ.) ...,2018-07-05 13:53:26,education,https://www.matichon.co.th/education/news_1029668,1029668


In [4]:
## drop column except for article
df.drop(columns=['date','headline','url','id'], inplace=True)

## tokenize
df['article_tokens'] = df['article'].apply(my_tokenize)

df

,article,category,article_tokens
0,(2 ธ.ค.58) หนังสือพิมพ์อัล ริยาดของทางการซาอุด...,foreign,"[2, ธ.ค., 58, หนังสือพิมพ์, อัล, ริยาด, ของ, ท..."
1,"""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",politics,"[บิ๊ก, ป้อม, แจง, ครม., มี, ความพยายาม, ยุยง, ..."
2,แม้จะทำหน้าที่ภรรยาที่ดีมาเฝ้าปอ – ทฤษฎี สหวงษ...,entertainment,"[แม้, จะ, ทำหน้าที่, ภรรยา, ที่, ดี, มา, เฝ้า,..."
3,กลายเป็นดาราหนุ่มเนื้อหอมแฟนคลับแน่น กระแสมาแร...,entertainment,"[กลายเป็น, ดารา, หนุ่ม, เนื้อ, หอม, แฟนคลับ, แ..."
4,แสดงดีจนเป็นที่ถูกอกถูกใจแฟนคลับ จนได้รับพวงมา...,entertainment,"[แสดง, ดี, จน, เป็นที่, ถูกอกถูกใจ, แฟนคลับ, จ..."
...,...,...,...
17104,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,politics,"[เมื่อ, วันที่, 5, ก.ค., ที่, สน., พญาไท, น.ส...."
17105,วันที่ 5 กรกฎาคม เจ้าหน้าที่ขุดทางระบายน้ำที่ด...,region,"[วันที่, 5, กรกฎาคม, เจ้าหน้าที่, ขุด, ทาง, ระ..."
17106,"สนช.ผ่าน พ.ร.บ.สงฆ์ 3 วาระรวด ""วิษณุ"" แจงสาระส...",politics,"[สนช, ผ่าน, พ.ร.บ., สงฆ์, 3, วาระ, รวด, วิษณุ,..."
17107,หมายเหตุ – จากกรณีที่ กระทรวงศึกษาธิการ (ศธ.) ...,education,"[หมายเหตุ, จาก, กรณี, ที่, กระทรวงศึกษาธิการ, ..."


# Fit model with `gensim`

how to use :
[https://radimrehurek.com/gensim/models/word2vec.html](https://radimrehurek.com/gensim/models/word2vec.html)

- input of `Word2Vec` must be **list of list of tokens**
- `vector_size` : Dimensionality of the word vectors (usually 100-300)
- `window` : Maximum distance between the current and predicted word within a sentence
- `min_count` : Ignores all words with total frequency lower than this
- `sg` : Training algorithm: 1 for skip-gram; otherwise CBOW. (skip-gram is recommended)
- `epoch` : Number of iterations (epochs) over the corpus

In [5]:
## fit
## epoch=30, it may take over 5 minutes
model = Word2Vec(sentences=df['article_tokens'], vector_size=100, window=5, min_count=3, sg=1, epochs=30)

# Word Embeddings

- `model.wv.most_similar(word, topn=xx)` gives the top N words with high cosine similarity 


In [6]:
model.wv.most_similar('อร่อย', topn=10)

[('รสชาติ', 0.8033337593078613),
 ('กลมกล่อม', 0.759904146194458),
 ('ลิ้มลอง', 0.7575984001159668),
 ('เมนู', 0.7486538290977478),
 ('ถูกปาก', 0.7146302461624146),
 ('ต้มยำ', 0.6957536935806274),
 ('จุใจ', 0.6873340606689453),
 ('ซี้ด', 0.6825014352798462),
 ('คุณหนู', 0.6794096827507019),
 ('โฮมเมด', 0.6754654049873352)]

- `model.wv.similarity(word1, word2)` calculates the cosine similarity of the 2 words

In [7]:
## antonym is also similar word
model.wv.similarity('ผู้ชาย', 'ผู้หญิง')

0.798946

- `model.wv` behaves like a dictionary
- e.g. `model.wv[word]` gives the vector of the word

In [8]:
model.wv['อร่อย']

array([-6.90128028e-01, -5.73293492e-02,  3.69323045e-02, -1.27124333e+00,
       -8.65098089e-02, -4.09606583e-02,  3.47794771e-01,  5.94089210e-01,
        1.56222284e-01, -4.75051105e-01,  1.36699881e-02, -8.80421340e-01,
       -3.51476282e-01,  9.74132195e-02,  6.15406811e-01, -7.93572545e-01,
       -6.17238641e-01, -4.84165341e-01, -4.52541485e-02, -2.17632592e-01,
        6.32575870e-01, -4.34085846e-01,  3.74199748e-01, -1.23329811e-01,
        2.12773457e-02, -2.75268018e-01,  3.35272551e-01,  4.45930421e-01,
       -1.25464925e-03, -5.72437286e-01,  1.41862631e-01,  3.74919713e-01,
        5.33240318e-01,  2.09685549e-01, -1.88794404e-01, -1.43954754e-01,
       -1.03247866e-01, -3.99757683e-01,  6.40755892e-01,  1.23744950e-01,
        1.82721719e-01,  4.75530922e-01,  3.28260869e-01, -4.49067146e-01,
        6.69365823e-01,  3.04452572e-02, -3.66540253e-01,  1.57527760e-01,
       -6.30566776e-02,  6.21690869e-01,  3.41537625e-01, -4.48221117e-01,
       -9.88482177e-01, -

- `model.wv.most_similar()` can add/subtract vectors
- use argument `positive` and `negative`, e.g. `model.wv.most_similar(positive=[w1, w2], negative=[w3])`


~~~python
'ปักกิ่ง' - 'จีน' + 'ญี่ปุ่น' : model.wv.most_similar(positive=['ปักกิ่ง', 'ญี่ปุ่น'], negative=['จีน'])
~~~

In [9]:
## ปักกิ่ง - จีน + ญี่ปุ่น
model.wv.most_similar(positive=['ปักกิ่ง', 'ญี่ปุ่น'], negative=['จีน'])

[('โตเกียว', 0.6826370358467102),
 ('กรุง', 0.630218505859375),
 ('เยรูซาเลม', 0.5580424666404724),
 ('มะนิลา', 0.5503596067428589),
 ('ฮาเนดะ', 0.5496883392333984),
 ('อาบูดาบี', 0.5293322205543518),
 ('เยอรมนี', 0.5223350524902344),
 ('โอซากา', 0.5204705595970154),
 ('จาการ์ตา', 0.5170543193817139),
 ('เบอร์ลิน', 0.5170120000839233)]

# save & load model

In [10]:
## save the model
## the same model is in `data` folder
model.save("data/word2vec_matichon.model")

In [11]:
## load pre-trained model 
model_loaded = Word2Vec.load("data/word2vec_matichon.model")
model_loaded.wv.most_similar('สวย')

[('เซ็กซี่', 0.6849587559700012),
 ('สวยงาม', 0.6509441137313843),
 ('ซิกซ์แพ็ก', 0.6408807039260864),
 ('สะบึม', 0.6297575235366821),
 ('น่ารัก', 0.628147542476654),
 ('เปล่งประกาย', 0.6210014820098877),
 ('สะพรั่ง', 0.6196946501731873),
 ('อึ๋ม', 0.6060378551483154),
 ('เชือดเฉือน', 0.5976887345314026),
 ('บิกินี่', 0.595427930355072)]

# Visualization

- go to https://projector.tensorflow.org/
- prepare 2 files:
    1. `tsv` file of all vectors (without index, without header)
    2. `tsv` file of labels (without index, with header)
- click "Load" and upload files
- it shows neighbor words in 3D plot

In [21]:
## make dataframe 
wv_df = pd.DataFrame(model.wv.vectors, columns=np.arange(1, 101)) # value of vectors
wv_df['word'] = model.wv.index_to_key # all vocabs
wv_df['count'] = wv_df['word'].apply(lambda x: model.wv.get_vecattr(x, 'count')) # get word count of each word
wv_df['freq/10M'] = (wv_df['count'] * 10000000 / wv_df['count'].sum()).round(1) # calculate word frequency in 10M words

wv_df.head()

,1,2,3,4,5,6,7,8,9,10,...,94,95,96,97,98,99,100,word,count,freq/10M
0,0.147107,0.205480,-0.080089,0.009322,-0.110583,-0.359767,0.186321,0.264107,-0.145018,0.053605,...,0.146914,0.328782,0.044164,-0.107805,-0.103107,0.094986,-0.065493,ที่,124850,202167.5
1,-0.076751,0.106975,-0.088567,-0.155876,0.131924,-0.413932,0.050694,0.405921,-0.200009,0.023447,...,-0.287021,0.205203,0.251952,0.006049,-0.087677,-0.002882,0.151624,และ,118744,192280.1
2,0.248766,0.236299,-0.139819,-0.158100,0.108476,-0.211760,0.100985,0.349802,-0.093654,0.041861,...,-0.271012,0.309146,0.211601,-0.102019,-0.161443,-0.143182,-0.138014,ใน,98174,158971.5
3,0.017607,0.330226,-0.128629,0.018312,0.143110,-0.360320,-0.297999,0.193926,-0.059721,-0.189001,...,-0.288652,0.388178,-0.151375,0.011995,0.012237,-0.119617,-0.043767,มี,84284,136479.6
4,0.009582,0.126914,0.096425,-0.209569,0.019580,-0.174211,-0.008858,0.744829,-0.203719,-0.405993,...,-0.447429,0.289066,0.165223,0.089111,0.048112,-0.004735,-0.147302,การ,74873,121240.6


In [22]:
## save to tsv file
## vector tsv files (without index, header)
wv_df.drop(columns=['word','count','freq/10M'], axis=1).to_csv('data/wv.tsv', sep='\t', index=False, header=False)

## label tsv files (with label `word` and `word frequency`)
wv_df[['word','freq/10M']].to_csv('data/wv_label.tsv', sep='\t', index=False)

![wv_visualization](image/wv_visualization.png)